In [1]:
from pycolator import *

idCol = 'SpecId'
scoreCol = 'NuXL:score'
excluded = ['SpecId', 'Label', 'ScanNr', 'Peptide', 'Proteins', 'FDR', 'q-val', 'Rank']

dSlow = readAndProcess('../Data/1-AChernev_080219_dir_HeLa_cyt_UCGA_slow.tsv', idCol, scoreCol)
p = percolator(dSlow, idCol, excluded, suppressLog = True)

file read
strings converted to numbers
q-values estimated
ranks computed


In [74]:
df = p

scores = [x for x in list(df.columns) if (x not in excluded)]

ixFalse = list(df[df.Label == 0].index)

falseTrain = df[scores].loc[ixFalse].values.tolist()
trueTrain = df[scores][(df['q-val'] <= 0.05) & (df.Label == 1)].values.tolist()

# compute training and response sets
train = falseTrain + trueTrain
classes = [0] * len(falseTrain) + [1] * len(trueTrain)

W = svm.LinearSVC(dual = False, C = 1, class_weight = {0:1,1:1}).fit(train, classes)
X = df[scores].values.tolist()
df['testScore'] = W.decision_function(X)
calcQ(df, 'testScore')
len(df[df['q-val'] <= 0.01])

5108